<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# リアルタイム レコメンデーション API 環境を構築する

このリファレンス アーキテクチャは、レコメンデーションシステム構築の完全なライフ サイクルを示しています。このシナリオでは、Azure Databricks を使用して、API として展開するレコメンデーション モデルのトレーニング、適切な Azure リソースの作成を説明します。Azure Cosmos DB、Azure Machine Learning と Azure Kubernetes Service を使用します。

このアーキテクチャは、製品、映画・番組、およびニュースのレコメンデーションを含む多くの一般的なレコメンデーション エンジン シナリオに適用することができます。
### アーキテクチャ
![architecture](https://recodatasets.z20.web.core.windows.net/images/reco-arch.png "Architecture")

**シナリオ**: とあるメディア企業は、自社のユーザーに映画やビデオのレコメンデーションを提供したいと考えています。パーソナライズされたレコメンデーションを提供することによって、クリックスルー率やサイトの利用率、ユーザー満足度の高い増加を通じて、いくつかのビジネス目標を満たそうとしています。

このリファレンスでは、訓練し、特定のユーザーに対してトップ 10 のレコメンドする映画を提供することが可能なリアルタイムのレコメンデーション サービス API をトレーニングし、展開します。

### コンポーネント
このアーキテクチャには、以下の主要なコンポーネントが含まれます:
* [Azure Databricks](https://docs.microsoft.com/ja-jp/azure/azure-databricks/what-is-azure-databricks)<sup>1)</sup> は、入力データの準備と、Spark クラスター上でレコメンデーション モデルのトレーニング開発環境として使用されます。Azure Databricks はまた、データの処理や機械学習のタスクを実行するためのノートブックで共同作業を行うためのインタラクティブなワークスペースを提供します。
* [Azure Kubernetes Service](https://docs.microsoft.com/ja-jp/azure/aks/intro-kubernetes)(AKS) は、Kubernetes クラスター上に機械学習モデルサービス API の展開と運用を行うために使用されます。AKS は、コンテナー化されたモデルをホストし、スループット要件を満たすためのスケーラビリティ、ID およびアクセスの管理、ログおよび状態監視を提供します。
* [Azure Cosmos DB](https://docs.microsoft.com/ja-jp/azure/cosmos-db/introduction) は、ユーザーごとにレコメンドされた映画のトップ 10 を格納するために使用される、グローバル分散型データベース サービスです。Azure Cosmos DB は指定されたユーザーのトップ 10 のレコメンド アイテムの読み取りに対して低レイテンシ (99 パーセンタイルにおいて 10 ms) で提供可能なため、このシナリオに最適です。
* [Azure Machine Learning](https://docs.microsoft.com/ja-jp/azure/machine-learning/) は、機械学習モデルのトラッキングと管理、スケーラブルな Azure Kubernetes Service環境にこれらのモデルをパッケージ化、展開するために使用するサービスです。

<sup>1) ここでは、Azure Databricks の使用例を示します。[SETUP](../../SETUP.md) にリストされているプラットフォームは、いずれも使用できます。</sup>


### 目次
0. [ファイルのインポート](#0-File-Imports)
1. [サービスの作成](#1-Service-Creation)
2. [トレーニング](#2-Training)
3. [運用化](#3.-Operationalize-the-Recommender-Service)

## セットアップ
Azure Databricks でこのノートブックを実行するには、リポジトリの[セットアップ手順](../../SETUP.md)の該当するセクションに従って Azure Databricks をセットアップし、このノートブックを Azure Databricks ワークスペースにインポートする必要があります ([こちら](https://docs.azuredatabricks.net/user-guide/notebooks/notebook-manage.html#import-a-notebook)の手順を参照してください)。

注意: このノートブックでは、**運用化** をサポートするために依存関係を追加する **必要** があります。詳細については、[SETUP](../../SETUP.md) を参照してください。

## 0 ファイルのインポート

In [1]:
import os
import sys
sys.path.append("../../")
import urllib

from azure.common.client_factory import get_client_from_cli_profile
import azure.mgmt.cosmosdb
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import Webservice, AksWebservice
from azureml.exceptions import WebserviceException
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.environment import SparkPackage
import pydocumentdb.document_client as document_client
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType

from reco_utils.common.timer import Timer
from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.dataset import movielens
from reco_utils.dataset.cosmos_cli import find_collection, read_collection, read_database, find_database
from reco_utils.dataset.download_utils import maybe_download
from reco_utils.dataset.spark_splitters import spark_random_split
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from reco_utils.common.notebook_utils import is_databricks

print("Azure SDK version:", azureml.core.VERSION)

Azure SDK version: 1.0.69


In [3]:
# 必要に応じて Spark セッションを開始する
if not is_databricks():
    cosmos_connector = (
        "https://search.maven.org/remotecontent?filepath=com/microsoft/azure/"
        "azure-cosmosdb-spark_2.3.0_2.11/1.3.3/azure-cosmosdb-spark_2.3.0_2.11-1.3.3-uber.jar"
    )
    jar_filepath = maybe_download(url=cosmos_connector, filename="cosmos.jar")
    spark = start_or_get_spark("ALS", memory="10g", jars=[jar_filepath])
    sc = spark.sparkContext
display(sc)

<SparkContext master=local[*] appName=ALS>

## 1 サービスの作成
デプロイするサブスクリプションに対して **サブスクリプション ID** を変更し、リソース名変数を設定します。

#### このノートブックで作成されるサービス:
1. [Azure ML Service](https://azure.microsoft.com/ja-jp/services/machine-learning-service/)
    1. [Azure ML Workspace](https://docs.microsoft.com/ja-jp/azure/machine-learning/concept-workspace)
    1. [Azure Application Insights](https://azure.microsoft.com/ja-jp/services/monitor/)
    1. [Azure Storage](https://docs.microsoft.com/ja-jp/azure/storage/common/storage-account-overview)
    1. [Azure Key Vault](https://azure.microsoft.com/ja-jp/services/key-vault/)    

1. [Azure Cosmos DB](https://azure.microsoft.com/ja-jp/services/cosmos-db/)
1. [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/ja-jp/services/kubernetes-service/)

**自分のサブスクリプション ID を追加**

In [4]:
# 自分のサブスクリプション ID を追加
subscription_id = ""

# ワークスペース名をセットする
workspace_name = "o16n-test"
resource_group = "{}-rg".format(workspace_name)

# Azure ML ワークスペースを自分のリージョンをセットする
location = "eastus"

# AzureML と Azure Kubernetes Service のプリフィックスを指定する
service_name = "mvl-als"

In [ ]:
# AzureML が Azure CLI ログイン資格情報を使用できるように、Azure CLI にログインします。
!az login

In [ ]:
# 必要に応じてサブスクリプションを変更する
!az account set --subscription {subscription_id}

In [ ]:
# アカウントのチェック
!az account show

In [9]:
# CosmosDB
# CosmosDB のaccount_nameは '_' を使用できず、31 文字未満である必要があります
account_name = "{}-ds-sql".format(workspace_name).replace("_", "-")[:31]
cosmos_database = "recommendations"
cosmos_collection = "user_recommendations_als"

# AzureML リソース名
model_name = "{}-reco.mml".format(service_name)
aks_name = "{}-aks".format(service_name)

In [10]:
# レコメンドする上位 k 個の項目
TOP_K = 10

# MovieLens のデータサイズを選択します: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [11]:
userCol = "UserId"
itemCol = "MovieId"
ratingCol = "Rating"

train_data_path = "train"
test_data_path = "test"

### 1.1 インポートまたは AzureML ワークスペースの作成
このコマンドは、AzureML ワークスペースが存在するかどうかを確認し、存在しない場合はワークスペースを作成します。

In [ ]:
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group, 
    location=location,
    exist_ok=True
)

### 1.2 レコメンデーション結果を保存する Cosmos DB を作成する

この手順でCosmosDB リソースを作成するには、多少時間がかかります。

In [13]:
# ユーザーが複数のサブスクリプションを持っている場合に、明示的に subscription_id を渡す
client = get_client_from_cli_profile(
    azure.mgmt.cosmosdb.CosmosDB,
    subscription_id=subscription_id
)

async_cosmosdb_create = client.database_accounts.create_or_update(
    resource_group,
    account_name,
    {
        'location': location,
        'locations': [{
            'location_name': location
        }]
    }
)
account = async_cosmosdb_create.result()

my_keys = client.database_accounts.list_keys(resource_group, account_name)
master_key = my_keys.primary_master_key
endpoint = "https://" + account_name + ".documents.azure.com:443/"

# DB クライアント
client = document_client.DocumentClient(endpoint, {'masterKey': master_key})

if not find_database(client, cosmos_database):
    db = client.CreateDatabase({'id': cosmos_database })
    print("Database created")
else:
    db = read_database(client, cosmos_database)
    print("Database found")

# コレクション オプション の作成
options = dict(offerThroughput=11000)

# コレクションの作成
collection_definition = {
    'id': cosmos_collection,
    'partitionKey': {'paths': ['/id'],'kind': 'Hash'}
}
if not find_collection(client, cosmos_database, cosmos_collection):
    collection = client.CreateCollection(
        db['_self'], 
        collection_definition,
        options
    )
    print("Collection created")
else:
    collection = read_collection(client, cosmos_database, cosmos_collection)
    print("Collection found")
    
dbsecrets = dict(
    Endpoint=endpoint, 
    Masterkey=master_key, 
    Database=cosmos_database, 
    Collection=cosmos_collection, 
    Upsert=True
)

Database created
Collection created


## 2 トレーニング

次に、[MovieLens](https://grouplens.org/datasets/movielens/) データセットで[最小二乗法モデル](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)をトレーニングします。

### 2.1 MovieLens データセットをダウンロードする

In [14]:
# 注: ALS 用のデータフレーム ベースの API は、現在、ユーザー ID とアイテム ID の整数のみをサポートしています。
schema = StructType(
    (
        StructField(userCol, IntegerType()),
        StructField(itemCol, IntegerType()),
        StructField(ratingCol, FloatType()),
    )
)

data = movielens.load_spark_df(spark, size=MOVIELENS_DATA_SIZE, schema=schema)
data.show()

100%|██████████| 4.81k/4.81k [00:00<00:00, 11.0kKB/s]


+------+-------+------+
|UserId|MovieId|Rating|
+------+-------+------+
|   196|    242|   3.0|
|   186|    302|   3.0|
|    22|    377|   1.0|
|   244|     51|   2.0|
|   166|    346|   1.0|
|   298|    474|   4.0|
|   115|    265|   2.0|
|   253|    465|   5.0|
|   305|    451|   3.0|
|     6|     86|   3.0|
|    62|    257|   2.0|
|   286|   1014|   5.0|
|   200|    222|   5.0|
|   210|     40|   3.0|
|   224|     29|   3.0|
|   303|    785|   3.0|
|   122|    387|   5.0|
|   194|    274|   2.0|
|   291|   1042|   4.0|
|   234|   1184|   2.0|
+------+-------+------+
only showing top 20 rows



### 2.2 データをトレーニングとテスト用に分割する
データを分割する方法は、ランダム、時系列、階層化など、それぞれが異なる実世界の評価のユースケースを優先する方法です。この例ではランダムに分割します – どのスプリッタを選択するかの詳細については、[このガイド](../01_prepare_data/data_split.ipynb)を参照してください。

In [15]:
train, test = spark_random_split(data, ratio=0.75, seed=42)
print("N train", train.cache().count())
print("N test", test.cache().count())

N train 75031
N test 24969


### 2.3 トレーニング データを使って ALS モデルをトレーニングする

映画の評価を予測するには、トレーニングセットの評価データをユーザーの明示的なフィードバックとして使用します。モデルの推定に使用されるハイパーパラメーターは、[このページ](http://mymedialite.net/examples/datasets.html)に基づいて設定されます。ほとんどの場合、ハイパーパラメーターを調べ、いくつかの基準に基づいて最適なセットを選択します。このプロセスの詳細については、詳細については、[こちら](../04_model_select_and_optimize/tuning_spark_als.ipynb)の詳細をご覧ください。

In [16]:
als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    alpha=0.1,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=True,
    userCol=userCol,
    itemCol=itemCol,
    ratingCol=ratingCol,
)

In [17]:
model = als.fit(train)

### 2.4 テストデータを利用してトップK個のレコメンデーションを取得する

映画おすすめのユースケースでは、ユーザーが評価したおすすめ映画は意味を成しません。したがって、評価された動画はレコメンデーション項目から削除されます。

これを実現するために、すべてのユーザーにすべてのムービーを推奨し、トレーニング データセットに存在するユーザーとムービーのペアを削除します。

In [18]:
# すべてのユーザーと項目のペアのクロス結合を取得し、それらをスコア付けします。
users = train.select(userCol).distinct()
items = train.select(itemCol).distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item)
dfs_pred.show()

+------+-------+----------+
|UserId|MovieId|prediction|
+------+-------+----------+
|   148|    148| 2.2560365|
|   463|    148|  2.936453|
|   471|    148| 3.8262048|
|   496|    148| 2.2901149|
|   833|    148| 1.7296925|
|   243|    148| 2.2667758|
|   392|    148| 2.4605818|
|   540|    148| 3.0631547|
|   623|    148| 3.1649487|
|   737|    148| 1.7344649|
|   858|    148| 1.8472893|
|   897|    148| 3.5229573|
|    31|    148| 1.9613894|
|   516|    148| 3.1411705|
|    85|    148| 2.2291098|
|   137|    148| 4.0498815|
|   251|    148| 3.2075853|
|   451|    148|  4.016654|
|   580|    148|  2.843738|
|   808|    148| 3.4666717|
+------+-------+----------+
only showing top 20 rows



In [19]:
# 表示された項目を削除します。
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    train.alias("train"),
    (dfs_pred[userCol]==train[userCol]) & (dfs_pred[itemCol]==train[itemCol]),
    how='outer'
)
top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train."+ratingCol].isNull()) \
    .select("pred."+userCol, "pred."+itemCol, "pred.prediction")

top_all.show()

+------+-------+----------+
|UserId|MovieId|prediction|
+------+-------+----------+
|     1|    587| 3.4595456|
|     1|    869|  2.967618|
|     1|   1208|  2.858056|
|     1|   1677| 2.9235902|
|     2|     80| 3.0129535|
|     2|    303| 3.0719132|
|     2|    472| 3.4143965|
|     2|    582|  4.877232|
|     2|    838|  1.529903|
|     2|    975| 2.9654517|
|     2|   1260|  3.252151|
|     2|   1325| 1.1417896|
|     2|   1381| 3.7900786|
|     2|   1530|  2.625749|
|     3|     22| 2.7082264|
|     3|     57| 2.5156925|
|     3|     89| 3.7927365|
|     3|    367| 2.7083492|
|     3|   1091| 1.5662774|
|     3|   1167| 3.2427955|
+------+-------+----------+
only showing top 20 rows



### 2.5 ALS のパフォーマンスを評価する

Precision@K、Recall@K、[MAP@K](https://en.wikipedia.org/wiki/Evaluation_measures_\(information_retrieval\) 、[nDCG@K](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)などのメトリックを使用して、モデルのパフォーマンスを評価します。推薦者を評価する指標の完全なガイドについては、[このガイド](../03_evaluate/evaluation.ipynb)を参照してください。

In [20]:
cols = {
    'col_user': userCol,
    'col_item': itemCol,
    'col_rating': ratingCol,
    'col_prediction': "prediction",
}

test.show()

+------+-------+------+
|UserId|MovieId|Rating|
+------+-------+------+
|     1|      2|   3.0|
|     1|      3|   4.0|
|     1|      4|   3.0|
|     1|     14|   5.0|
|     1|     17|   3.0|
|     1|     27|   2.0|
|     1|     29|   1.0|
|     1|     35|   1.0|
|     1|     36|   2.0|
|     1|     51|   4.0|
|     1|     52|   4.0|
|     1|     54|   3.0|
|     1|     56|   4.0|
|     1|     60|   5.0|
|     1|     64|   5.0|
|     1|     69|   3.0|
|     1|     77|   4.0|
|     1|     83|   3.0|
|     1|     85|   3.0|
|     1|     88|   4.0|
+------+-------+------+
only showing top 20 rows



In [21]:
# ランキング指標の評価
rank_eval = SparkRankingEvaluation(
    test, 
    top_all, 
    k=TOP_K,
    **cols
)

print(
    "Model:\tALS",
    "Top K:\t%d" % rank_eval.k,
    "MAP:\t%f" % rank_eval.map_at_k(),
    "NDCG:\t%f" % rank_eval.ndcg_at_k(),
    "Precision@K:\t%f" % rank_eval.precision_at_k(),
    "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n'
)

Model:	ALS
Top K:	10
MAP:	0.003698
NDCG:	0.034331
Precision@K:	0.039343
Recall@K:	0.014976


In [22]:
# ランキング指標の評価
prediction = model.transform(test)
rating_eval = SparkRatingEvaluation(
    test, 
    prediction, 
    **cols
)

print(
    "Model:\tALS rating prediction",
    "RMSE:\t%.2f" % rating_eval.rmse(),
    "MAE:\t%f" % rating_eval.mae(),
    "Explained variance:\t%f" % rating_eval.exp_var(),
    "R squared:\t%f" % rating_eval.rsquared(), sep='\n'
)

Model:	ALS rating prediction
RMSE:	0.95
MAE:	0.740282
Explained variance:	0.289807
R squared:	0.285394


### 2.6 モデルを保存する

In [23]:
(model
 .write()
 .overwrite()
 .save(model_name))

## 3. Recommender サービスの運用化
モデルが望ましいパフォーマンスで構築されると、リアルタイム サービスで使用される REST エンドポイントとして実行するように運用が可能になります。[Azure Cosmos DB](https://azure.microsoft.com/ja-jp/services/cosmos-db/)、[Azure Machine Learning](https://azure.microsoft.com/ja-jp/services/machine-learning-service/)、および [Azure Kubernetes Service](https://docs.microsoft.com/ja-jp/azure/aks/intro-kubernetes)を利用して、レコメンデーション サービスを運用します。

### 3.1 Cosmos DB でレコメンデーションのルックアップを作成する

まず、モデルで予測される各ユーザーの上位 10 件の推奨事項が、Cosmos DB にルックアップ テーブルとして格納されます。実行時に、サービスは、Cosmos DB に事前計算され、格納されたとして、Top-10 の推奨事項を返します：

In [24]:
recs = model.recommendForAllUsers(10)
recs_topk = recs.withColumn("id", recs[userCol].cast("string")) \
    .select("id", "recommendations." + itemCol)
recs_topk.show()

+---+--------------------+
| id|             MovieId|
+---+--------------------+
|471|[745, 1540, 244, ...|
|463|[64, 190, 1286, 3...|
|833|[1192, 179, 1524,...|
|496|[320, 1589, 262, ...|
|148|[1512, 718, 793, ...|
|540|[958, 1512, 1368,...|
|392|[1643, 1449, 1512...|
|243|[285, 251, 1405, ...|
|623|[390, 1643, 173, ...|
|737|[856, 60, 61, 151...|
|897|[1368, 958, 320, ...|
|858|[1154, 1129, 853,...|
| 31|[1203, 1245, 889,...|
|516|[745, 694, 1512, ...|
|580|[1368, 958, 1589,...|
|251|[1203, 1449, 253,...|
|451|[1368, 1019, 958,...|
| 85|[1643, 1449, 511,...|
|137|[1368, 1643, 958,...|
|808|[1512, 867, 1367,...|
+---+--------------------+
only showing top 20 rows



In [25]:
# データを Cosmos DB に保存
(recs_topk.coalesce(1)
 .write
 .format("com.microsoft.azure.cosmosdb.spark")
 .mode('overwrite')
 .options(**dbsecrets)
 .save())

### 3.2 Azure Azure Machine Learning の構成

次に、Azure Machine Learning を使用して、モデル スコアリング イメージを作成し、スケーラブルなコンテナー化されたサービスとして Azure Kubernetes Service にデプロイします。これを実現するには、**スコアリング スクリプト** を作成する必要があります。スクリプトでは、Cosmos DB に呼び出しを行い、入力ユーザー ID を推奨する上位 10 の映画を検索します。

In [26]:
score_sparkml = """
import json
import pydocumentdb.document_client as document_client

def init(local=False):
    global client, collection
    try:
        client = document_client.DocumentClient('{endpoint}', dict(masterKey='{key}'))
        collection = client.ReadCollection(collection_link='dbs/{database}/colls/{collection}')
    except Exception as e:
        collection = e

def run(input_json):
    try:
        # Query them in SQL
        id = str(json.loads(json.loads(input_json)[0])['id'])
        query = dict(query='SELECT * FROM c WHERE c.id = "' + id +'"')
        options = dict(partitionKey=str(id))
        document_link = 'dbs/{database}/colls/{collection}/docs/' + id
        result = client.ReadDocument(document_link, options);  
    except Exception as e:
        result = str(e)
    return json.dumps(str(result))
""".format(key=dbsecrets['Masterkey'], 
           endpoint=dbsecrets['Endpoint'], 
           database=dbsecrets['Database'], 
           collection=dbsecrets['Collection'])

# Python 文字列の有効性をテストする
exec(score_sparkml)

with open("score_sparkml.py", "w") as file:
    file.write(score_sparkml)

モデルの登録:

In [27]:
mymodel = Model.register(
    model_path=model_name,  # これはローカル ファイルへのポイント
    model_name=model_name,  # これは登録されているモデル名
    description="AML trained model",
    workspace=ws
)

print(mymodel.name, mymodel.description, mymodel.version)

Registering model mvl-als-reco.mml
mvl-als-reco.mml AML trained model 1


### 3.3 AKS にサービスとしてモデルを展開

#### 3.3.1 モデルの環境を作成:

In [28]:
env = Environment(name='sparkmlenv')

# ベース イメージとして Microsoft/mmlspark のパブリック イメージを指定する
env.docker.base_image="microsoft/mmlspark:0.15"

pip = [
    'azureml-defaults', 
    'numpy==1.14.2', 
    'scikit-learn==0.19.1', 
    'pandas', 
    'pydocumentdb'
]

# 推論に必要な依存関係を追加する
env.python.conda_dependencies = CondaDependencies.create(pip_packages=pip)
env.inferencing_stack_version = "latest"

# Spark パッケージの追加
env.spark.precache_packages = True
env.spark.repositories = ["https://mmlspark.azureedge.net/maven"]
env.spark.packages= [
    SparkPackage("com.microsoft.ml.spark", "mmlspark_2.11", "0.15"),
    SparkPackage("com.microsoft.azure", artifact="azure-storage", version="2.0.0"),
    SparkPackage(group="org.apache.hadoop", artifact="hadoop-azure", version="2.7.0")
]

#### 3.3.2 コンテナーを実行する AKS クラスターを作成
クラスタサイズによっては、20～30 分かかる場合があります。

In [29]:
# クラスタが存在していないことを確認
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    # 既定の構成を使用してクラスターを作成 (カスタマイズするパラメーターも提供可能)
    prov_config = AksCompute.provisioning_configuration()
    aks_target = ComputeTarget.create(
        workspace=ws, 
        name=aks_name, 
        provisioning_configuration=prov_config
    )
    aks_target.wait_for_completion(show_output = True)
    print(aks_target.provisioning_state)
    # エラー ログを確認するには、print(aks_target.provisioning_errors) を実行

Creating.......................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded


#### 3.3.3 コンテナー イメージを AKS に展開:

In [30]:
# 環境、スコアリング スクリプトを使用して推論構成を作成する
inference_config = InferenceConfig(
    environment=env,
    entry_script="score_sparkml.py"
)

# Web サービスの構成の設定 (ここでは App Insights でデフォルトを使用)
aks_config = AksWebservice.deploy_configuration(enable_app_insights=True)

# 単一コマンドを使用した Web サービスの作成
try:
    aks_service = Model.deploy(
        workspace=ws,
        models=[mymodel],
        name=service_name,
        inference_config=inference_config,
        deployment_config=aks_config,
        deployment_target=aks_target
    )
    aks_service.wait_for_deployment(show_output=True)
except WebserviceException:
    # 既存のサービスを取得
    aks_service = Webservice(ws, name=service_name)
    print("Retrieved existing service")

Running....................................................................................................................
SucceededAKS service creation operation finished, operation "Succeeded"


### 3.4 AKS モデル サービスを呼び出す
デプロイ後、サービスはユーザー ID で呼び出すことができます 。次のスクリプトは、レコメンデーション サービス API を呼び出し、指定されたユーザー ID の結果を表示する方法を示しています：

In [31]:
import json

scoring_url = aks_service.scoring_uri
service_key = aks_service.get_keys()[0]

input_data = '["{\\"id\\":\\"496\\"}"]'.encode()

req = urllib.request.Request(scoring_url, data=input_data)
req.add_header("Authorization","Bearer {}".format(service_key))
req.add_header("Content-Type","application/json")

with Timer() as t: 
    with urllib.request.urlopen(req) as result:
        res = result.read()
        resj = json.loads(
            # json オブジェクトに解析するためのクリーンアップ
            res.decode("utf-8")
            .replace("\\", "")
            .replace('"', "")
            .replace("'", '"')
        )
        print(json.dumps(resj, indent=4))
    
print("Full run took %.2f seconds" % t.interval)

{
    "MovieId": [
        320,
        1589,
        262,
        1344,
        958,
        889,
        1368,
        645,
        919,
        1137
    ],
    "id": "496",
    "_rid": "34hEAIe9pterAQAAAAAACA==",
    "_self": "dbs/34hEAA==/colls/34hEAIe9ptc=/docs/34hEAIe9pterAQAAAAAACA==/",
    "_etag": "6d006b74-0000-0100-0000-5f25f0550000",
    "_attachments": "attachments/",
    "_ts": 1596321877
}
Full run took 0.05 seconds


## Appendix - AzureML を使用したリアルタイム スコアリング

以前のセルでは、Cosmos DB を利用して、リアルタイム・サービスの推奨結果をキャッシュしました。それ以外にも、展開したモデルを使用してオンデマンドでレコメンデーション結果を生成することもできます。次のスクリプトは、登録されたモデルを読み込み、レコメンデーションのために使用します：

* *score_sparkml.py*
    ```
    import json
    import os
    from pyspark.ml.recommendation import ALSModel

    # Note, set `model_name`, `userCol`, and `itemCol` defined earlier.
    model_name = "mvl-als-reco.mml"
    userCol = "UserId"
    itemCol = "MovieId"

    def init(local=False):
        global model

        # Load ALS model.
        model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_name)
        model = ALSModel.load(model_path)

    def run(input_json):
        js = json.loads(json.loads(input_json)[0])
        id = str(js['id'])
        k = js.get('k', 10)

        # Use the model to get recommendation.
        recs = model.recommendForAllUsers(k)
        recs_topk = recs.withColumn('id', recs[userCol].cast("string")).select(
            'id', "recommendations." + itemCol
        )
        result = recs_topk[recs_topk.id==id].collect()[0].asDict()

        return json.dumps(str(result))
    ```

* AKS モデル サービスを呼び出す
    ```
    # Get a recommendation of 10 movies
    input_data = '["{\\"id\\":\\"496\\",\\"k\\":10}"]'.encode()

    req = urllib.request.Request(scoring_url, data=input_data)
    req.add_header("Authorization","Bearer {}".format(service_key))
    req.add_header("Content-Type","application/json")
    
    ...
    ```